# Price Targets

## Summary

The analysis can be run below to regress mean returns or total returns over a period against variables from the estimate immediately preceding the period.

The immediate result is that for most windows only the variable `sd_pct`, which is standard deviation as a fraction of mean price target estimate, emerges as statistically significant, with a positive coefficient. So greater uncertainty in estimates typically results in greater returns. There are a few different ways one could try to justify this. However, the coefficient magnitude is very low when considered alongside typical `sd_pct` and response values, so I do not think this is a thread worth spending time on. The `sd_pct` significance dissipates for shorter windows

For longer time horizons `low_sd` (difference between low and mean as pct of standard deviation of estimates) shows up as significant. Again however the magnitude of this value is very low and values close to 0 are sometimes in the 95% confidence interval. While this result has a clear interpretation, it is probably best left bookmarked as a signal we may want to combine with something much stronger, if at all. 



In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../src')
from utils.load_bucket_prices import (
    load_bucket_prices, load_bucket_factset_estimates)

plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (15, 5)

In [3]:
start = '2012-01-01'
end = '2021-02-18'

all_data = load_bucket_prices('..', start, end)
before_size = all_data.shape[0]
all_data = all_data.dropna(how='all')
print(f'Dropped {before_size - all_data.shape[0]} all-na rows')

close_prices = all_data['adj_close']
open_prices = all_data['adj_open']
close_prices.columns.name = 'ticker'
open_prices.columns.name = 'ticker'

Found existing data file. Reading...
Data read from: ../data/raw/prc_scorecard_single_ticker_2012-01-01_2021-02-18_factset.csv
Dropped 1 all-na rows


/Users/mhandley/Envs/proteus-financial3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (273,368,370,388,399) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
end = '2021-03-15'
# estimates = get_rolling_estimates(close_prices.columns[100:], start, end)

estimates = load_bucket_factset_estimates('..', 'PRICE_TGT', start, end)

Found existing data file. Reading...
Data read from: ../data/raw/estimates_PRICE_TGT_scorecard_single_ticker_2012-01-01_2021-03-15_factset.csv


In [6]:
estimates_range = estimates['fiscalEndDate'].min(), estimates['fiscalEndDate'].max()
prices_range = close_prices.index.min(), close_prices.index.max()

print(f'Estimates range: {estimates_range}')
print(f'Prices range: {prices_range}')

print('Fixing estimates to price range')
estimates = estimates[
    (estimates['fiscalEndDate'] >= prices_range[0]) & \
    (estimates['fiscalEndDate'] <= prices_range[1])]

Estimates range: (Timestamp('2011-12-31 00:00:00'), Timestamp('2021-01-31 00:00:00'))
Prices range: (Timestamp('2011-12-30 00:00:00'), Timestamp('2021-02-18 00:00:00'))
Fixing estimates to price range


In [15]:
from evaluation.price_targets import price_targets_test_df

df = price_targets_test_df(close_prices, estimates, 10, 90)

df[[
    'price_realised', 'price_at_estimate', 'mean', 'from_current',
    'miss', 'mean_return', 'realised_diff', 'fiscalEndDate',
    'total_return']]

price_realised  price_at_estimate        mean  \
ticker estimateDate                                                  
AAL    2014-04-14         33.146797          31.459278   43.866667   
       2014-07-14         37.983055          40.519024   51.218750   
       2014-10-13         37.720177          27.011848   51.583333   
       2015-04-14         48.600975          45.021637   68.250000   
       2015-07-14         39.167873          40.014600   56.966667   
...                             ...                ...         ...   
ZS     2019-02-19         57.530000          49.990000   44.100000   
       2019-05-21         71.960000          76.090000   60.898462   
       2020-02-11         50.410000          60.920000   61.944444   
       2020-05-19        106.170000          75.240000   67.315789   
       2020-08-31        130.100000         143.340000  115.863636   

                     from_current      miss  mean_return  realised_diff  \
ticker estimateDate                                                       
AAL    2014-04-14        1.394395  0.755626     0.002441       0.053641   
       2014-07-14        1.264067  0.741585     0.000600      -0.062587   
       2014-10-13        1.909656  0.731247     0.006286       0.396431   
       2015-04-14        1.515938  0.712102    -0.000596       0.079503   
       2015-07-14        1.423647  0.687558     0.000363      -0.021160   
...                           ...       ...          ...            ...   
ZS     2019-02-19        0.882176  1.304535     0.004999       0.150830   
       2019-05-21        0.800348  1.181639    -0.004193      -0.054278   
       2020-02-11        1.016816  0.813794     0.007202      -0.172521   
       2020-05-19        0.894681  1.577193     0.007689       0.411085   
       2020-08-31        0.808313  1.122872     0.004346      -0.092368   

                    fiscalEndDate  total_return  
ticker estimateDate                              
AAL    2014-04-14      2014-03-31      0.221548  
       2014-07-14      2014-06-30      0.021654  
       2014-10-13      2014-09-30      0.700831  
       2015-04-14      2015-03-31     -0.075887  
       2015-07-14      2015-06-30      0.010226  
...                           ...           ...  
ZS     2019-02-19      2019-01-31      0.476695  
       2019-05-21      2019-04-30     -0.353003  
       2020-02-11      2020-01-31      0.687623  
       2020-05-19      2020-04-30      0.833998  
       2020-08-31      2020-07-31      0.368006  

[2688 rows x 9 columns]

## 90d Returns

In [26]:
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

fit = smf.ols(
    formula='mean_return ~ low_sd + hi_sd + from_current + sd_pct',
    data=df,
    missing='raise') \
    .fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            mean_return   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     13.74
Date:                Wed, 31 Mar 2021   Prob (F-statistic):           4.28e-11
Time:                        11:06:51   Log-Likelihood:                 12181.
No. Observations:                2688   AIC:                        -2.435e+04
Df Residuals:                    2683   BIC:                        -2.432e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0014      0.000      3.655      0.000       0.001       0.002
low_sd           0.0002   8.06e-05      2.546      0.011    4.72e-05       0.000
hi_sd          1.63e-05   9.48e-05      0.172      0.863      -0.000       0.000
from_current    -0.0003      0.000     -1.293      0.196      -0.001       0.000
sd_pct           0.0042      0.001      5.867      0.000       0.003       0.006
==============================================================================
Omnibus:                      353.853   Durbin-Watson:                   1.499
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1977.328
Skew:                           0.490   Prob(JB):                         0.00
Kurtosis:                       7.086   Cond. No.                         46.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
influence = OLSInfluence(fit)
cooks, pvals = influence.cooks_distance
np.min(pvals)

0.9162357168599626

## 10d, 30d returns

In [13]:
df = price_targets_test_df(close_prices, estimates, 10, 30)

fit = smf.ols(
    formula='mean_return ~ low_sd + hi_sd + from_current + sd_pct',
    data=df,
    missing='raise') \
    .fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            mean_return   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     10.94
Date:                Wed, 31 Mar 2021   Prob (F-statistic):           8.45e-09
Time:                        10:51:28   Log-Likelihood:                 10781.
No. Observations:                2797   AIC:                        -2.155e+04
Df Residuals:                    2792   BIC:                        -2.152e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -2.861e-05      0.001     -0.040      0.968      -0.001       0.001
low_sd           0.0003      0.000      1.777      0.076   -2.85e-05       0.001
hi_sd            0.0002      0.000      1.038      0.299      -0.000       0.001
from_current     0.0006      0.000      1.630      0.103      -0.000       0.001
sd_pct           0.0072      0.001      5.255      0.000       0.004       0.010
==============================================================================
Omnibus:                      247.454   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1631.738
Skew:                          -0.010   Prob(JB):                         0.00
Kurtosis:                       6.742   Cond. No.                         45.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
df = price_targets_test_df(close_prices, estimates, 10, 10)

fit = smf.ols(
    formula='mean_return ~ low_sd + hi_sd + from_current + sd_pct',
    data=df,
    missing='raise') \
    .fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            mean_return   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.781
Date:                Wed, 31 Mar 2021   Prob (F-statistic):              0.130
Time:                        10:51:48   Log-Likelihood:                 8732.1
No. Observations:                2859   AIC:                        -1.745e+04
Df Residuals:                    2854   BIC:                        -1.742e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0013      0.002      0.808      0.419      -0.002       0.004
low_sd           0.0004      0.000      1.278      0.201      -0.000       0.001
hi_sd            0.0003      0.000      0.854      0.393      -0.000       0.001
from_current     0.0003      0.001      0.305      0.760      -0.001       0.002
sd_pct           0.0047      0.003      1.560      0.119      -0.001       0.011
==============================================================================
Omnibus:                      262.818   Durbin-Watson:                   1.889
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1091.018
Skew:                           0.367   Prob(JB):                    1.23e-237
Kurtosis:                       5.936   Cond. No.                         45.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 252 day returns

In [18]:
df = price_targets_test_df(close_prices, estimates, 10, 252)

fit = smf.ols(
    formula='mean_return ~ low_sd + hi_sd + from_current + sd_pct',
    data=df,
    missing='raise') \
    .fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            mean_return   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     21.07
Date:                Wed, 31 Mar 2021   Prob (F-statistic):           4.23e-17
Time:                        10:52:09   Log-Likelihood:                 12465.
No. Observations:                2460   AIC:                        -2.492e+04
Df Residuals:                    2455   BIC:                        -2.489e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0012      0.000      5.337      0.000       0.001       0.002
low_sd           0.0002   4.91e-05      3.241      0.001    6.29e-05       0.000
hi_sd        -6.888e-05   5.76e-05     -1.196      0.232      -0.000     4.4e-05
from_current    -0.0002      0.000     -1.278      0.201      -0.000    8.39e-05
sd_pct           0.0037      0.000      7.558      0.000       0.003       0.005
==============================================================================
Omnibus:                      454.101   Durbin-Watson:                   0.880
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2506.832
Skew:                           0.758   Prob(JB):                         0.00
Kurtosis:                       7.707   Cond. No.                         51.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 90d Total returns

In [23]:
df = price_targets_test_df(close_prices, estimates, 10, 90)

fit = smf.ols(
    formula='total_return ~ low_sd + hi_sd + from_current + sd_pct',
    data=df,
    missing='raise') \
    .fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           total_return   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     15.21
Date:                Wed, 31 Mar 2021   Prob (F-statistic):           2.66e-12
Time:                        10:53:47   Log-Likelihood:                -428.65
No. Observations:                2688   AIC:                             867.3
Df Residuals:                    2683   BIC:                             896.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.1227      0.040      3.041      0.002       0.044       0.202
low_sd           0.0194      0.009      2.205      0.028       0.002       0.037
hi_sd            0.0053      0.010      0.510      0.610      -0.015       0.026
from_current    -0.0388      0.022     -1.760      0.079      -0.082       0.004
sd_pct           0.4890      0.078      6.277      0.000       0.336       0.642
==============================================================================
Omnibus:                     1733.509   Durbin-Watson:                   1.566
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            52086.191
Skew:                           2.577   Prob(JB):                         0.00
Kurtosis:                      23.940   Cond. No.                         46.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""